# Design Fair Demo

Welcome to the Emotion Watchers design fair! Here, we'll take a look into the end-to-end process of testing the emotion detection model, from data collection to inference. 

<p align="center">
  <img src="../images/transparent_logo.png" width="80" height="80">
</p>


## Data Collection

If not already done, be sure to collect the facial expression videos and pupillometry data of the participant. 

In order for the data processing step to work with the given commands, ensure that:

<input type='checkbox' > The video is saved in `C:\Users\meria\Pictures\Camera Roll`  <br>
<input type='checkbox' > The video name follows the structure of `<partipant_id>_<emotion>.mp4` <br>
<input type='checkbox' > The pupillometry data should be saved in `C:\Users\meria\Documents\GazePoint\result` <br>
<input type='checkbox' > The pupillometry file should be named `<participant_id>_all_gaze.csv` <br>
<input type='checkbox' > The output_path (`\data`) should be cleared <br>

## Data Processing

Once the data has been collected, it must be processed. 

The facial video is converted to a sequence of images and cropped to include only the face. The face can be detected either using the [Haar Cascade model](https://www.datacamp.com/tutorial/face-detection-python-opencv) or a UI. These images are then classified by the emotion and resized to be interpreted by the model.

The pupillometry data is processed using MATLAB to remove outliers, such as blinking. Then the average of the left and right pupil dilations is outputted. Since the pupillometry data is discrete and has gaps after removing outliers, we created a continuous function from the data points using a [cubic spline](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.CubicSpline.html) interpolation technique.

To process the data, run the command below:



In [9]:
!python ..\data_processing\data_processing\process_data.py "C:\Users\meria\Pictures\Camera Roll" ..\data C:\Users\meria\Documents\GazePoint\result

Error using table (line 283)
Incorrect number of arguments.

Error in process_data (line 50)
    segmentsTable = table(segmentStart, segmentEnd, segmentName);

Caused by:
    You might have intended to create a one-row table with the character vector '7.mp4' as one of its variables. To store text data in a table, use a string array or a cell array of character vectors rather than character arrays. Alternatively, create a cell array with one row, and convert that to a table using CELL2TABLE.

Traceback (most recent call last):
  File "c:\Users\meria\source\repos\emotion-watchers\data_processing\data_processing\process_data.py", line 72, in <module>
    process_data(
  File "c:\Users\meria\source\repos\emotion-watchers\data_processing\data_processing\process_data.py", line 22, in process_data
    pupil.process_data(pupil_dir, plot_matlab, plot_result)
  File "c:\users\meria\source\repos\emotion-watchers\data_processing\data_processing\pupil\process_data.py", line 153, in process_data
   

## Models

To integrate facial expressions and pupillometry into emotion prediction, two separate models were initially developed and later combined into an ensemble. The facial model was trained on cropped and grayscale facial images, while the pupillometry model was trained on sequences of pupil dilations. The fusion model is utilized solely for testing purposes.

### Facial Model

The facial model processes cropped facial images and offers predictions in two modes: binary and multiclass.

- **Binary Mode**: Emotions are categorized as either positive or negative.
- **Multiclass Mode**: Emotions are classified into several categories, including anger, fear, joy, fun, calm, happy, and sad.

The current implementation utilizes the multiclass mode.

#### Architecture

The facial model employs the following architecture:

- **Convolutional Layers**: These layers are responsible for extracting features from the input images.
- **Linear Layers**: Following the feature extraction, linear layers make predictions based on the extracted features.


<p align="center">
  <img src="../images/face_model.png" height=200>
</p>
<p align="center">
  <i> Figure: Architecture of the Facial Model </i>
</p>

### Pupillometry Model

The pupillometry model focuses on binary classification at present. However, it can be extended to multiclass classification by incorporating binary predictions into corresponding emotions.

#### Classification Mode

- **Binary Classification**: The model currently operates in binary classification mode.

#### Architecture

The model prioritizes analyzing changes in pupil dilation rather than focusing solely on absolute dilation values. The architecture comprises:

- **LSTM Layers**: These layers retain memory over a sequence of dilations, capturing temporal dynamics.
- **Convolutional Layers**: Responsible for extracting features from pupil dilation sequences.
- **Linear Layers**: These layers make predictions based on the extracted features.

<p align="center">
  <img src="../images/pupil_model.png" height=200>
</p>
<p align="center">
  <i> Figure: Architecture of the Pupillometry Model </i>
</p>

### Fusion Model



In [8]:
!python ..\models\models\fusion.py C:\Users\meria\Documents\GazePoint\result ..\data

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 32)        9

2024-04-08 13:33:13.824582: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.

2024-04-08 13:33:21.730385: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE SSE2 SSE3 SSE4.1 SSE4.2 AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "c:\Users\meria\source\repos\emotion-watchers\models\models\fusion.py", line 151, in <module>
    create_confusion_matrix(labels, predictions, classes)
  File "c:\Users\meria\source\repos\emotion-watchers\models\models\fusion.py", line 97, in create_confusion_matrix
    disp.plot()
  File "c:\Users\meria\AppData\Local\Pro